<a href="https://colab.research.google.com/github/LordHandLee/HonCSC499_detect_falls_soccer/blob/main/MLP_base_fallen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob

import matplotlib.pyplot as plt
import cv2
from skimage import data,io
import matplotlib.image as mpimg

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers.experimental import preprocessing
#from autokeras.utils import data_utils

from sklearn.model_selection import train_test_split
import pathlib

In [3]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:

data_dir = "/content/drive/MyDrive/honor_thesis/image_depo/"
# data_dir = pathlib.Path(data_dir).with_suffix('')
# for i in data_dir.glob("*"):
#   print(i)
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.3)
width = 150
height = 150
trainDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='training')
valDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='validation')
feature_vector_length = 150*150*3
input_shape = (feature_vector_length,)

Found 611 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [5]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=30)

In [6]:

# Create the model
model = Sequential()
model.add(Reshape((67500,), input_shape=(150,150,3)))
model.add(Dense(350, input_shape=input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Configure the model and start training
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=[keras.metrics.BinaryAccuracy()])

history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500
39/39 [==============================] - 303s 8s/step - loss: 40.7229 - binary_accuracy: 0.5205 - val_loss: 8.8178 - val_binary_accuracy: 0.5000
Epoch 2/500
39/39 [==============================] - 3s 70ms/step - loss: 4.8339 - binary_accuracy: 0.5483 - val_loss: 0.8313 - val_binary_accuracy: 0.5885
Epoch 3/500
39/39 [==============================] - 3s 73ms/step - loss: 0.9216 - binary_accuracy: 0.5843 - val_loss: 0.7245 - val_binary_accuracy: 0.5846
Epoch 4/500
39/39 [==============================] - 3s 68ms/step - loss: 0.5913 - binary_accuracy: 0.6612 - val_loss: 0.7357 - val_binary_accuracy: 0.5923
Epoch 5/500
39/39 [==============================] - 3s 67ms/step - loss: 0.6427 - binary_accuracy: 0.6465 - val_loss: 0.6933 - val_binary_accuracy: 0.6038
Epoch 6/500
39/39 [==============================] - 3s 68ms/step - loss: 0.6045 - binary_accuracy: 0.6825 - val_loss: 0.7901 - val_binary_accuracy: 0.4885
Epoch 7/500
39/39 [==============================] - 3s 71ms/st

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 67500)             0         
                                                                 
 dense (Dense)               (None, 350)               23625350  
                                                                 
 dense_1 (Dense)             (None, 50)                17550     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 23642951 (90.19 MB)
Trainable params: 23642951 (90.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
